In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [3]:
loader = PyPDFLoader('../docs/reglas_wh40k.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)  
chunks = splitter.split_documents(docs)


In [26]:
llm_gpt = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.getenv("OPENAI_API_KEY"))
gemma = ChatGroq(model="gemma2-9b-it", api_key=os.getenv("GROQ_API_KEY"))

In [3]:
chunks[0]

Document(metadata={'source': '../docs/reglas_wh40k.pdf', 'page': 0}, page_content='Reglas básicas\n“ Estamos rodeados desde todos los flancos por viles \nalienígenas depredadores y la sedición nos corroe \ndesde dentro; en esta hora tan oscura, lo mejor que \npodemos hacer es confiar en nuestro armamento y \nrezar a nuestros dioses.”\n- Skolak a’Trellar IV, Comandante Imperial')

In [11]:
pagina = chunks[0].page_content
map_prompt = PromptTemplate.from_template("Resume esto:\n\n{page_content}")
resumen_chain = map_prompt | gemma
resumen_chain.invoke({'page_content': pagina})


AIMessage(content='En medio de una batalla contra invasores alienígenas y la amenaza interna de la rebelión, el Comandante Imperial Skolak a’Trellar IV anima a su tropa a confiar en su poder y a buscar la protección divina. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 89, 'total_tokens': 141, 'completion_time': 0.094545455, 'prompt_time': 0.004016365, 'queue_time': 0.161869601, 'total_time': 0.09856182}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--83b7552a-92e9-40cc-b878-c570f18fd86e-0', usage_metadata={'input_tokens': 89, 'output_tokens': 52, 'total_tokens': 141})

In [24]:
llm = ChatGroq(model="meta-llama/llama-guard-4-12b", api_key=os.getenv("GROQ_API_KEY"))
map_prompt = PromptTemplate(
    input_variables=["page_content"],
    template=(
        "Eres un experto en Warhammer 40K.\n"
        "Resume este extracto, resaltando reglas clave y ejemplos prácticos:\n\n"
        "{page_content}\n"
    )
)
combine_prompt = PromptTemplate(
    input_variables=["summaries"],
    template=(
        "Con los siguientes resúmenes parciales, elabora un resumen final "
        "detallado de las reglas de Warhammer 40K.\n"
        "La estructura del resumen tiene que incluir una breve introducción explicando en que consiste y objetivos y despues IMPORTANTE explicar  las fases del turno:"
        "Debe tener al menos 1 párrafo para cubrir cada una de las fases de Movimiento, Disparo, Carga, . Mete todo el contenido que consideres importante.  Si es necesario hacer el resumen más largo que 5 párrafos no dudes en hacerlo."
        "Combate y Moral:\n\n"
        "{summaries}\n"
    )
)

summary_chain = load_summarize_chain(
    llm_gpt,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    map_reduce_document_variable_name="page_content",
    combine_document_variable_name="summaries",
    token_max=15000,
    verbose=True
)

final_summary = summary_chain.run(chunks)
final_summary



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Eres un experto en Warhammer 40K.
Resume este extracto, resaltando reglas clave y ejemplos prácticos:

Reglas básicas
“ Estamos rodeados desde todos los flancos por viles 
alienígenas depredadores y la sedición nos corroe 
desde dentro; en esta hora tan oscura, lo mejor que 
podemos hacer es confiar en nuestro armamento y 
rezar a nuestros dioses.”
- Skolak a’Trellar IV, Comandante Imperial

Prompt after formatting:
Eres un experto en Warhammer 40K.
Resume este extracto, resaltando reglas clave y ejemplos prácticos:

¡Bienvenidos a las Reglas básicas de 
Warhammer 40,000! Las siguientes 
páginas contienen todo lo que necesitas 
saber para librar gloriosas batallas a través 
de la galaxia en guerra del 41.er Milenio.
Warhammer 40,000 es un juego de guerra 
sobre tablero en el que los jugadores 
controlan ejércitos de miniaturas Citadel 
e intentan derrotar a sus oponentes con 
u

'En Warhammer 40K, los jugadores se sumergen en una batalla estratégica donde despliegan sus ejércitos, controlan puntos clave y eliminan a sus enemigos para alcanzar la victoria. El objetivo principal es acumular puntos de victoria controlando marcadores de objetivo y eliminando fuerzas enemigas.\n\nEl juego se desarrolla en diversas fases: movimiento, disparo, carga, combate y moral. En la fase de movimiento, las unidades se desplazan estratégicamente por el campo de batalla para controlar objetivos y posicionarse de forma ventajosa. En la fase de disparo, las unidades realizan ataques a distancia contra enemigos visibles dentro de su rango de tiro, teniendo en cuenta las características de sus armas.\n\nDurante la fase de combate, las unidades pueden realizar ataques cuerpo a cuerpo contra enemigos cercanos, resolviendo los ataques de forma alternada y determinando las heridas infligidas. En la fase de moral, las unidades deben superar un chequeo para evitar huir del campo de batall

In [25]:
final_summary

'En Warhammer 40K, los jugadores se sumergen en una batalla estratégica donde despliegan sus ejércitos, controlan puntos clave y eliminan a sus enemigos para alcanzar la victoria. El objetivo principal es acumular puntos de victoria controlando marcadores de objetivo y eliminando fuerzas enemigas.\n\nEl juego se desarrolla en diversas fases: movimiento, disparo, carga, combate y moral. En la fase de movimiento, las unidades se desplazan estratégicamente por el campo de batalla para controlar objetivos y posicionarse de forma ventajosa. En la fase de disparo, las unidades realizan ataques a distancia contra enemigos visibles dentro de su rango de tiro, teniendo en cuenta las características de sus armas.\n\nDurante la fase de combate, las unidades pueden realizar ataques cuerpo a cuerpo contra enemigos cercanos, resolviendo los ataques de forma alternada y determinando las heridas infligidas. En la fase de moral, las unidades deben superar un chequeo para evitar huir del campo de batall

In [26]:
with open('../docs/resumen_w40k.txt', "w") as f:
    f.write(final_summary)

In [7]:
embedding_model = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory="../docs/vectordb/"
)

In [8]:
vectordb.add_documents(chunks)

['d4df6fec-7c20-450c-97dc-bbedd10b8e52',
 '6d60e977-635c-42f9-b0ff-424d991f7ac5',
 '075eec12-985e-43ec-812e-bd6349bbeeeb',
 'e6a0dd1f-17f9-4cdf-825a-8be31ee2ba04',
 '9e55496b-abd6-4cdd-b473-953c9d4a60ff',
 '79af5d30-e802-4ac9-9711-f1a4296a2b2d',
 'f8fb4cd8-cab5-44cf-af55-3a44da01fdc1',
 '09ba6fa3-b428-4e29-b457-caff475e34bc',
 '19840a1c-60ed-41d1-b56d-ea253deca202',
 '830fc072-94ce-4196-be7a-b3fe89074f6e',
 'd810afab-70ac-4a78-84a0-cefd131d122f',
 'e1fd63a9-bce3-4ca4-bf50-e59b089555e3',
 '37dd2161-5171-4fa4-a16e-4d82f4297e4c',
 '72ce6b51-ff57-4ada-897d-312388aa2d5d',
 'd3562834-34ad-407c-9ff2-02792b1f09e3',
 '785f3116-0660-476f-9046-0dcfe572c712',
 '749eef9b-55e0-48e5-8c4f-7c268eba1a80',
 '528543c6-e4f1-45ab-aca7-88484a4e8214',
 '5de6c732-ccf8-48f2-940b-73286b4b8df0',
 '7baca6ad-c66e-4400-80b7-7f0996e0fc77',
 '4c22ddc0-dd75-46db-943f-09b30e8839d1',
 'a2e41f34-93d8-4491-a120-bf21ae25f153',
 'f0bc5f97-6da6-48dd-8441-7f8a8d1d0950',
 '845b5167-e8df-4273-922c-99f5ba80356a',
 '0c54f0ff-f6ff-

In [12]:
retriever = vectordb.as_retriever(search_type='mmr', search_kwargs = {'k': 8})
retrieved_docs = retriever.invoke("cuantas armas puede disparar una unidad en la fase de disparo?")

In [32]:

retriever = vectordb.as_retriever(search_type='mmr', search_kwargs = {'k': 6})
question = "cuantas armas puede disparar una unidad en la fase de disparo?"
retrieved_docs = retriever.invoke("cuantas armas puede disparar una unidad en la fase de disparo?")

context = ''
for r in retrieved_docs:
    context += r.page_content
context

template_message = "Eres un experto en partidas de Warhammer 40k. Estás para ayudar a que jugadores nuevos aprendan a jugar partidas. Tu tarea es resolver las dudas que tengan y explicarlas \
    de manera que la entiendan fácilmente y no se queden con más dudas. Para responder, te pido que hables como hablaría un verdadero Adeptus Astartes leal al emperador. Te voy a dar parte de las reglas para que te sirva como contexto para formular tu respuesta. \
    - \tAquí tienes la pregunta: {question}\
    - \tEl contexto de las reglas: {context}"

template = ChatPromptTemplate.from_template(template=template_message)

llm = ChatGroq(model="meta-llama/llama-guard-4-12b", api_key=os.getenv("GROQ_API_KEY"))
llama_3 = ChatGroq(model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

chain_dudas = template | llama_3 | StrOutputParser()
chain_dudas.invoke({'question': question, 'context': context})


'Hermano, te saludo en el nombre del Emperador. Me alegra que estés dispuesto a aprender las reglas de la guerra en el universo de Warhammer 40k. La pregunta que me haces es fundamental para entender cómo se desarrolla una partida.\n\nEn cuanto a la cantidad de armas que puede disparar una unidad en la fase de disparo, la respuesta es simple: una unidad puede disparar con todas las armas a distancia que estén equipadas sus miniaturas, siempre y cuando estén dentro del alcance y visibilidad del blanco elegido.\n\nSegún las reglas, "cada vez que una unidad dispare, antes de resolver ningún ataque, debes elegir qué unidades enemigas serán blanco de todas las armas a distancia con las que quieres que ataquen las miniaturas de la unidad que dispara". Esto significa que una unidad puede disparar con todas sus armas a distancia, siempre y cuando estén dentro del alcance y visibilidad del blanco elegido.\n\nSin embargo, es importante tener en cuenta que algunas armas tienen restricciones espec

In [36]:
from pydantic import BaseModel, Field
from typing import Literal


class router_atrtibutes(BaseModel):
    tipo_pregunta : Literal['General', 'Especifica'] = Field(description="Indica si la pregunta es de caracter general si pregunta un resumen sobre las reglas de warhammer 40k o específica si es una pregunta concreta")
    workflow: Literal['audio', 'texto'] = Field(description="Indica si debe responder en texto o en audio")

router_message = "Eres un experto en interpretar y clasificar mensajes. Esto es necesario para poder redirigir el flujo de trabajo al asistente de dudas sobre las reglas de Warhammer 40k. Vas a recibir un mensaje de un usuario haciendote un comentario o una pregunta sobre las reglas de warhammer 40K\
    Necesito que analices lo siguiente: \n\
      - Tipo de pregunta: solo puedes devolver 'General' o 'Especifica'. En caso de ser una pregunta que sea de caracter general , por ejemplo un resumen de las reglas devolveras 'General'. Por el contrario si la pregunta es más detallada devolverás 'Específica'.\n\
      - Workflow: solo puedes devolver 'texto' o 'audio'. Esto indica como se debe de responder. Por norma general debe ser 'texto'. Elige solo 'audio' si el usuario lo pide expresamente. \n\
     "

router_prompt = ChatPromptTemplate.from_messages(
    [("system", router_message), MessagesPlaceholder(variable_name="messages")]
)

router_model = llama_3.with_structured_output(router_atrtibutes)

router_chain = router_prompt | router_model

response = router_chain.invoke({'messages':[ "Me puedes hacer un breve resumen de las reglas. Me gustaría oír tu voz"]})

response



router_atrtibutes(tipo_pregunta='General', workflow='audio')